###For Diabetic retinopathy

In [2]:
import numpy as np
import os
import cv2
#import sklearn
import tensorflow as tf
import itertools
import csv
import keras
import time

current_date = time.strftime("%d_%m_%Y_")

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)


Using TensorFlow backend.


In [4]:
#import keras and sklearn layers here
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda,Activation,Dropout
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.convolutional import MaxPooling2D as Mpool2D
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as KTF
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.activations import relu
from keras.layers import Flatten, Dense, Lambda,Cropping2D,Activation,Dropout,ELU
#from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.regularizers import l2,l1
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.regularizers import l2,l1
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.layers import Input, merge, Dropout, Dense, Flatten, Activation
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from random import shuffle


In [8]:
import numpy as np
#import pandas as pd
import os
import cv2
#import sklearn
import itertools
import keras
import time
keras.__version__

#import keras and sklearn layers here
from keras.models import load_model
from keras import applications
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda,Activation,Dropout
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.convolutional import MaxPooling2D #as Mpool2D
from keras.layers.convolutional import AveragePooling2D as AvgPool2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as KTF
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import Flatten, Dense, Lambda,Cropping2D,Activation,Dropout,ELU
#from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import regularizers
from keras.regularizers import l2,l1
#from utils.model_utils import *

#os.getcwd()

In [9]:
import keras as k
k.__version__

'2.2.2'

In [27]:

TRAINED_MODEL_FILE_NAME = "path_to_model/DR-epoch-{epoch:2d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.h5"

#Read train and validation data
train_samples = []
validation_samples = []


samples1 = []
samples2 = []
path1 = "Path to train"
listing1 = os.listdir(path1)  
for file1 in listing1:
    samples1.append(file1)


path2 = "Path to validation"
listing2 = os.listdir(path2)  
for file2 in listing2:
    samples2.append(file2)


num_classes = 2
print("len(train_samples):", len(samples1))
print("len(validation_samples):", len(samples2))


len(train_samples): 449
len(validation_samples): 133


In [ ]:
from random import shuffle
train_samples = list()
validation_samples = list()

for root, directories, files in os.walk(path1):
    for filename in files:
        train_samples.append(filename)

        
for root, directories, files in os.walk(path2):
    for filename in files:
        validation_samples.append(filename)

print("Before Shuffiling Train  " + train_samples[20])
shuffle(train_samples)
print("After Shuffiling  Train  " + train_samples[20])

print("Before Shuffiling Val    " + validation_samples[20])
shuffle(validation_samples)
print("After Shuffiling  Val    " + validation_samples[20])

In [30]:
#train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, batch_size,src):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        #shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
                
            images = []
            labels = []
            for batch_sample in batch_samples:
                #name = batch_sample.split('_')[1]
                #print("name:",batch_sample)
                image = cv2.imread(src+batch_sample)
                
                image = cv2.resize(image, (300,300))
                #plt.imshow(image)
                image = image.reshape(300,300,3)
                label = batch_sample.split('_')[0]
                if(label == "Diabetic_retinopathy"):
                    label = 0
                if(label == "healthy"):
                    label = 1
                
                
                images.append(image)
                labels.append(label)
            #print(labels)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(labels)
            y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
            yield sklearn.utils.shuffle(X_train, y_train)


In [31]:
#X_train,y_train = sklearn.utils.shuffle(X_train,y_train)
train_generator = generator(samples1, batch_size=8,src = path1)
validation_generator = generator(samples2, batch_size=8,src = path2)

In [58]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [61]:
x = (300,300,3)
model = Sequential()
model.add(Conv2D(32, (3, 3),activation="relu",input_shape=(300, 300, 3), data_format="channels_last", padding="same", kernel_initializer="glorot_normal", name="conv_1_1"))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3),activation="relu",input_shape=(300, 300, 3), data_format="channels_last", padding="same", kernel_initializer="glorot_normal", name="conv_1_4"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last", name="maxpool_1_1"))
model.add(Conv2D(64, (1, 1), activation="relu",data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_1_2"))
model.add(BatchNormalization())
model.add(Conv2D(64, (1, 1),activation="relu", data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_1_3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last", name="maxpool_1_2"))
model.add(Dropout(0.3, name="dropout_1_1"))
model.add(Conv2D(128, (3, 3),activation="relu", data_format="channels_last", padding="same", kernel_initializer="glorot_normal", name="conv_2_1"))
model.add(BatchNormalization())
model.add(Conv2D(128, (1, 1),activation="relu", data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_2_2"))
model.add(BatchNormalization())
model.add(Conv2D(128, (1, 1),activation="relu", data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_2_3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last", name="maxpool2_1"))
model.add(Conv2D(256, (3, 3),activation="relu", data_format="channels_last", padding="same", kernel_initializer="glorot_normal", name="conv_3_1"))
model.add(BatchNormalization())
model.add(Conv2D(256, (1, 1),activation="relu", data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_3_2"))
model.add(BatchNormalization())
model.add(Conv2D(256, (1, 1),activation="relu", data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_3_3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last", name="maxpool_3_1"))
model.add(Dropout(0.5, name="dropout_3_1"))
model.add(Conv2D(512, (3, 3), activation="relu",data_format="channels_last", padding="same", kernel_initializer="glorot_normal", name="conv_4_1"))
model.add(BatchNormalization())
model.add(Conv2D(512, (1, 1),activation="relu", data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_4_2"))
model.add(BatchNormalization())
model.add(Conv2D(512, (1, 1),activation="relu", data_format="channels_last", padding="valid", kernel_initializer="glorot_normal", name="conv_4_3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last", name="maxpool_4_1"))
model.add(Dropout(0.3, name="dropout_4_1"))
model.add(Conv2D(3, (1, 1),activation="relu", padding="valid", kernel_initializer="glorot_normal", data_format="channels_last", name="conv_5_1"))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D(data_format="channels_last",name="global_avg_pooling"))
model.add(Dense(3,name="FC_5_1"))  
model.add(Activation('softmax', name="softmax_5_1"))

In [62]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
batch_normalization_80 (Batc (None, 300, 300, 32)      128       
_________________________________________________________________
conv_1_4 (Conv2D)            (None, 300, 300, 32)      9248      
_________________________________________________________________
maxpool_1_1 (MaxPooling2D)   (None, 150, 150, 32)      0         
_________________________________________________________________
conv_1_2 (Conv2D)            (None, 150, 150, 64)      2112      
_________________________________________________________________
batch_normalization_81 (Batc (None, 150, 150, 64)      256       
_________________________________________________________________
conv_1_3 (Conv2D)            (None, 150, 150, 64)      4160      
__________

In [35]:

checkpoint = keras.callbacks.ModelCheckpoint(filepath=TRAINED_MODEL_FILE_NAME)
model.fit_generator(train_generator, steps_per_epoch= (len(samples1)/4),validation_data=validation_generator, validation_steps=(len(samples2)/4),epochs=20, callbacks=[checkpoint])

Epoch 1/20
113/112 [==============================] - 1237s 11s/step - loss: 0.4367 - acc: 0.8562 - auc_roc: 0.8743 - val_loss: 0.5631 - val_acc: 0.6842 - val_auc_roc: 0.8992
Epoch 2/20
113/112 [==============================] - 1212s 11s/step - loss: 0.2624 - acc: 0.9380 - auc_roc: 0.9035 - val_loss: 0.6437 - val_acc: 0.8045 - val_auc_roc: 0.9172
Epoch 3/20
113/112 [==============================] - 1180s 10s/step - loss: 0.1670 - acc: 0.9701 - auc_roc: 0.9217 - val_loss: 0.5060 - val_acc: 0.7293 - val_auc_roc: 0.9325
Epoch 4/20
113/112 [==============================] - 1180s 10s/step - loss: 0.1283 - acc: 0.9790 - auc_roc: 0.9391 - val_loss: 0.1671 - val_acc: 0.9323 - val_auc_roc: 0.9493
Epoch 5/20
113/112 [==============================] - 1180s 10s/step - loss: 0.1221 - acc: 0.9790 - auc_roc: 0.9553 - val_loss: 0.0240 - val_acc: 1.0000 - val_auc_roc: 0.9616
Epoch 6/20
113/112 [==============================] - 1181s 10s/step - loss: 0.0958 - acc: 0.9801 - auc_roc: 0.9663 - val_los

In [15]:
model.to_json()

'{"keras_version": "2.0.5", "backend": "tensorflow", "class_name": "Sequential", "config": [{"class_name": "Lambda", "config": {"output_shape_type": "raw", "output_shape": null, "batch_input_shape": [null, 224, 224, 3], "name": "lambda_1", "dtype": "float32", "trainable": true, "function_type": "lambda", "function": ["\\u00e3\\u0001\\u0000\\u0000\\u0000\\u0000\\u0000\\u0000\\u0000\\u0001\\u0000\\u0000\\u0000\\u0002\\u0000\\u0000\\u0000C\\u0000\\u0000\\u0000s\\f\\u0000\\u0000\\u0000|\\u0000\\u0000d\\u0001\\u0000\\u001bd\\u0002\\u0000\\u0018S)\\u0003Ng\\u0000\\u0000\\u0000\\u0000\\u0000\\u00e0o@g\\u0000\\u0000\\u0000\\u0000\\u0000\\u0000\\u00e0?\\u00a9\\u0000)\\u0001\\u00da\\u0001xr\\u0001\\u0000\\u0000\\u0000r\\u0001\\u0000\\u0000\\u0000\\u00fa\\u001e<ipython-input-9-28f918963b71>\\u00da\\b<lambda>\\u0002\\u0000\\u0000\\u0000s\\u0000\\u0000\\u0000\\u0000", null, null], "arguments": {}}}, {"class_name": "Conv2D", "config": {"padding": "same", "filters": 64, "bias_constraint": null, "kern